In [7]:
import os
import tempfile
import shutil
from functools import reduce

In [2]:
species = 'lumbricus_rubellus'
assembly = 'wcLumRube1.1' # can be found on the Ensembl project page
gff3_file = '/home/vonkamp/gwdg_owncloud/lumbricus_integrate_koala/Lumbricus_rubellus_-GCA_945859605.1-2023_03-genes.gff3'
unmasked_genome = '/scratch/vonkamp/lumbricus_rubellus/ensembl_data/Lumbricus_rubellus-GCA_945859605.1-unmasked.fa'

In [12]:
species = 'lumbricus_terrestris'
assembly = 'wcLumTerr1.1' # can be found on the Ensembl project page
gff3_file = '/scratch/vonkamp/lumbricus_terrestris/Lumbricus_terrestris-GCA_949752735.1-2023_11-genes.gff3'
unmasked_genome = '/scratch/vonkamp/lumbricus_terrestris/Lumbricus_terrestris-GCA_949752735.1-unmasked.fa'

In [8]:
biocyc_input = os.path.join(tempfile.tempdir, species+'_biocyc_pf_fa')
if os.path.exists(biocyc_input):
    shutil.rmtree(biocyc_input) # remove old files
os.mkdir(biocyc_input)
print("BioCyc input will be in:", biocyc_input)

BioCyc input will be in: /tmp/lumbricus_terrestris_biocyc_pf_fa


In [11]:
contig_count = 0
contig_file = None
with open(unmasked_genome) as fasta_file, \
     open(os.path.join(biocyc_input, 'genetic-elements.dat'), 'w') as ge_file:
    for line in fasta_file:
        idx = line.find(assembly)
        if idx >= 0:
            contig = line[idx:line.find(':', idx + len(assembly) + 1)]
            ge_file.write(f"ID\t{contig.replace('.', '_').replace(':', '-')}\nNAME\t{contig}\n")
            try:
                int(contig.split(':')[1])
            except ValueError:
                ge_file.write("TYPE\t:CONTIG\n")
                contig_count += 1
            else:
                ge_file.write("TYPE\t:CHRSM\n")
                print("Chromosome", contig)
            ge_file.write(f"CIRCULAR?\tN\nANNOT-FILE\t{contig}.pf\nSEQ-FILE\t{contig}.fa\n//\n")
            if contig_file is not None:
                contig_file.close()
            contig_file = open(os.path.join(biocyc_input, contig+'.fa'), 'w')
        contig_file.write(line)
contig_file.close()
print(contig_count, "non-chromosomal contigs.")

Chromosome wcLumTerr1.1:1
Chromosome wcLumTerr1.1:2
Chromosome wcLumTerr1.1:3
Chromosome wcLumTerr1.1:4
Chromosome wcLumTerr1.1:5
Chromosome wcLumTerr1.1:6
Chromosome wcLumTerr1.1:7
Chromosome wcLumTerr1.1:8
Chromosome wcLumTerr1.1:9
Chromosome wcLumTerr1.1:10
Chromosome wcLumTerr1.1:11
Chromosome wcLumTerr1.1:12
Chromosome wcLumTerr1.1:13
Chromosome wcLumTerr1.1:14
Chromosome wcLumTerr1.1:15
Chromosome wcLumTerr1.1:16
Chromosome wcLumTerr1.1:17
Chromosome wcLumTerr1.1:18
343 non-chromosomal contigs.


The BlastKOALA service needs to be run via its web interface (https://www.kegg.jp/blastkoala/) with e.g. Lumbricus_rubellus_-GCA_945859605.1-2023_03-pep.fa as input (it is currently necessary to split this file into two because it exceeds to numer of allowed proteins). The (merged) result is then placed in BlastKOALA/user_ko_definition.txt.

In [13]:
import re
def parse_user_ko_definition(filename: str, score=False) -> dict:
    ec_pattern = re.compile("[\s:][\d-]+\.[\d-]+\.[\d-]+\.[\d-]+") # match multiple and also partial EC numbers
    id2kegg_ec = dict()
    with open(filename) as file:
        for line in file:
            line = line.split("\t")
            match = ec_pattern.findall(line[2])
            if len(match) > 0:
                name = line[2][0:line[2].find("[EC:")].split(";")[1].strip()
                ec_number = set([m[1:] for m in match])
                if score:
                    s = float(line[3])
                    id2kegg_ec[line[0][:-2]] = (ec_number, name, s)
                else:
                    id2kegg_ec[line[0][:-2]] = (ec_number, name)
    return id2kegg_ec
ensembl_id2kegg_ec = parse_user_ko_definition(os.path.join(species, 'BlastKOALA', 'user_ko_definition.txt'))
print(len(ensembl_id2kegg_ec)) # number of predicted proteins that have an EC number
print(len(reduce(set.union, (ec[0] for ec in ensembl_id2kegg_ec.values())))) # number of distinct EC numbers found

5269
1232


#!genome-build  wcLumRube1.1


In [13]:
# this section relies particluarities of Ensembl GFF files
section_begin = "###"
gene_id_pattern = re.compile("ID=gene:(\w+)")
gene_name_pattern = re.compile("Name=(\w+)")
description_pattern = re.compile("description=.+\]")
cds_id_pattern = re.compile("ID=CDS:\w+")
region_pattern = re.compile("ID=region")
cds_id2function = dict()
new_section = False
in_gene_section = False
gene_count = 0
pf_file = None
current_gene_id = ''
current_gene_startbase = 0
current_gene_endbase = 0
current_cds_id = None
cds_segments = []
def write_transcript():
    global current_cds_id
    global cds_segments
    global cds_id2function
    if current_cds_id is not None:
        # !! if a real gene exists this will be used as ID by PathwayTools and the ID ist not recorded in the DB !!
        # pf_file.write(f"ID\t{current_gene_id}\nNAME\t{current_cds_id if current_gene_name is None else current_gene_name}\nPRODUCT-TYPE\tP\n") # LRULOC
        pf_file.write(f"ID\t{current_gene_id}\nNAME\t{current_gene_id if current_gene_name is None else current_gene_name}\nPRODUCT-TYPE\tP\nPRODUCT-ID\t{current_cds_id}\n") # LRULOC
        pf_file.write(f"STARTBASE\t{current_gene_startbase}\nENDBASE\t{current_gene_endbase}\n")
        for (s,e) in cds_segments:
            pf_file.write(f"CODING-SEGMENT\t{s}-{e}\n")

        if current_gene_description and current_cds_id not in cds_id2function:
            cds_id2function[current_cds_id] = current_gene_description # for later use
        ec_number = ensembl_id2kegg_ec.get(current_cds_id, None) # KEGG EC gets precedent over ENSEMBL function
        if ec_number:
            if len(ec_number[1]) > 0:
                for fn in ec_number[1].split(" / "):
                    pf_file.write(f"FUNCTION\t{fn}\n")
            elif current_gene_description: # in case only Architect identified an EC number...
                pf_file.write(f"FUNCTION\t{current_gene_description}\n") # ...use ENSEMBL function prediction as product description
            for ec in ec_number[0]:
                pf_file.write(f"EC\t{ec}\n")
        elif current_gene_description:
            pf_file.write(f"FUNCTION\t{current_gene_description}\n")
        pf_file.write("//\n")
    current_cds_id = None
    cds_segments.clear()

with open(gff3_file) as file:
    for line in file:
        region = region_pattern.search(line)
        if region:
            contig = assembly+line[region.span()[1]:line.find(";", region.span()[1])]
            if pf_file is not None:
                pf_file.close()
            pf_file = open(os.path.join(biocyc_input, contig+'.pf'), 'w')
            continue
        if line.startswith(section_begin):
            write_transcript()
            in_gene_section = False
            new_section = True
            current_gene_description = None
        elif new_section:
            new_section = False
            columns = line.split("\t")
            if columns[2] == "gene" and "biotype=protein_coding" in columns[8]:
                in_gene_section = True
                gene_count += 1
                current_gene_id = gene_id_pattern.search(columns[8])
                current_gene_id = current_gene_id.groups()[0]
                current_gene_name = gene_name_pattern.search(columns[8])
                if current_gene_name:
                    current_gene_name = current_gene_name.groups()[0]
                if columns[6] == '+':
                    current_gene_startbase = columns[3]
                    current_gene_endbase = columns[4]
                elif columns[6] == '-':
                    current_gene_startbase = columns[4]
                    current_gene_endbase = columns[3]
                else:
                    raise
                description = description_pattern.search(columns[8])
                if description:
                    current_gene_description = description.group(0)
                    current_gene_description = current_gene_description[12:current_gene_description.index("[")-1]
        elif in_gene_section:
            columns = line.split("\t")
            cds_id = cds_id_pattern.search(columns[8])
            if cds_id: # (part of) coding sequence
                cds_id = cds_id.group(0)[7:]
                if current_cds_id is None:
                    current_cds_id = cds_id
                elif cds_id != current_cds_id:
                    write_transcript()
                    current_cds_id = cds_id
                if columns[6] == '+':
                    cds_segments.append((columns[3], columns[4]))
                elif columns[6] == '-':
                    cds_segments.append((columns[4], columns[3]))
                else:
                    raise        
pf_file.close()
print(gene_count)

26609


NCBI Taxonomy IDs:
Lumbricus rubellus: 35632
Lumbricus terrestris: 6398

With PathoLogic component of PathwayTools:
1. Create new database, do not enter the replicon editor at the end of this step
2. Copy genetic-elements.dat file from the BioCyc input directory to pathway_tools_config_data/vonkamp/ptools-local/pgdbs/user/lrucyc/1.0/input
3. Run automated build